## Sampling and optimizing binding sites on a set of water clusters


```Prerequisite: qcfractal, psi4, geometric```

This tutorial will explain to you the procedure to compute a BE energy distribution on a amorphized set 
of water clusters. In the previous tutorial we learned how to create a OptimizationDatase. For this tutorial we assume that you already learned how to set up a [QCFractal server](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/setup_server.html) and spin up a [QCFractal manager](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/managers.html) on your server. The same instructions are valid if you want to contribute to the BEEP database, the only thing that changes is the URL of the server. 

So at this point you have already created a collection with a set of amporphised water clusters. We will use the set consisting of 12 water molecules constructed in the previous tutorial.

Before sampling the cluster, the geometry of the water cluster and in this example the methanol molecule to be sampled need to be optimized. For this pruprose we need to spin up a simple qcfractal-manager called basic.yaml which is contained in qcfractal_manager folder in this repository. Simply write in a terminal whithin your qcfractal enviornment `qcfractal-manager -c basic.yaml --verify False -v`. You can also use nohup to run it in the background. Once the manager is running we can send the optimizations:

In [58]:
import qcfractal.interface as ptl
client = ptl.FractalClient(address="localhost:7777", verify=False)
client.list_collections()

tagline
collection          name                       
OptimizationDataset pentamer_collection    None
                    pre_ch3oh_w12_1        None
                    small_molecules        None
                    water_12               None

In [39]:
ds_w12= client.get_collection("OptimizationDataset", "water_12")
ds_sm  = client.get_collection("OptimizationDataset", "small_molecules")

Once the collections are loaded we can add a specification with the details of the optimization in form of a
python dictionary:

In [9]:
spec = add_spec = {'name': 'blyp-d3bj_def2-svp',
        'description': 'Geometric + Psi4/blyp-d3bj/def2-svp.',
        'optimization_spec': {'program': 'geometric', 'keywords': None},
        'qc_spec': {'driver': 'gradient',
        'method': 'blyp-d3bj',
        'basis': 'def2-svp',
        'keywords': None,
        'program': 'psi4'}}
ds_w12.add_specification(**add_spec,overwrite=True)
ds_w12.save()
ds_sm.add_specification(**add_spec,overwrite=True)
ds_sm.save()

'2'

Finally we send the optimization to the manager we started with the tag basic.

In [10]:
ds_w12.compute('blyp-d3bj_def2-svp', tag='basic')

5

In [11]:
ds_sm.compute('blyp-d3bj_def2-svp', tag='basic')

1

We can check the progress of the optimitaztion with the `status()` method:

In [32]:
ds_sm.status(collapse = False)

,blyp-d3bj_def2-svp
ch3oh,COMPLETE


In [40]:
ds_w12.status(collapse = False)

,blyp-d3bj_def2-svp
w12_1,COMPLETE
w12_2,COMPLETE
w12_3,COMPLETE
w12_4,COMPLETE
w12_5,INCOMPLETE


In [57]:
client.delete_collection("OptimizationDataset", 'ch3oh_w12_1')
#ds_w12.get_record(wat_cluster, opt_lot).get_final_molecule()

Once all initial molecules are fully optimized, we can start the sampling procedure. It consists of three steps, **sampling the water clusters**, **computing the BSSE corrected binding energy for each unique binding site** and **computing the Hessian** for the binding site of one of the sampled clusters to obtain a zero-point vibrational energy correction for the binding energy calculation. For each one of these three steps there is a dedicated CLI tool. In this tutorial we will show how to use the first CLI for the sampling procedure.

The first step is to sample the surface of the water clusters that span the model surface. With this purpose, `launch_sampling` creates two OptimizationDataSets:  One that contains initial sampled structures (with the prefix *pre*) and a second one where the unique optimized structures are stored after the initial optimization at the sampling level of theory. After the initial optimization is done, the geometry of the binding sites is refined using a HF-3C/MINIX level of theory. Nonetheless the geometries can also be further optimized (e.g. at a DFT hybrid or meta-hyrbid GGA level of theory) as they are contained in a Dataset. 
For the sampling procedure we developed a option-specific command line interface `launch_sampling.py`. It can be found in the scripts folder of this module. 

```
Usage: python launch_sampling.py [options]

A command line interface to sample the surface of a set of water clusters (stored in a 
QCFractal DataSet)  with a small molecule or atom. This CLI is part
of the Binding Energy Evaluation Platform (BEEP).


Options:
  -h, --help            show this help message and exit
  --client_address=CLIENT_ADDRESS
                        The URL address and port of the QCFractal server
                        (default: localhost:7777)
  --molecule=MOLECULE   The name of the molecule to be sampled (from a
                        QCFractal OptimizationDataSet collection)
  --surface_model_collection=SURFACE_MODEL_COLLECTION
                        The name of the collection with the set of water
                        clusters (dafault: ASW_22)
  --small_molecule_collection=SMALL_MOLECULE_COLLECTION
                        The name of the collection containing molecules or
                        radicals (dafault: Small_molecules)
  --molecules_per_round=MOLECULE_PER_ROUND
                        Number of molecules to be optimized each round
                        (Default = 10)
  --sampling_shell=SAMPLING_SHELL
                        The shell size of sampling space (Default = 1.5
                        Angstrom)
  --maximal_binding_sites=MAXIMAL_BINDING_SITES
                        The maximal number of binding sites per cluster
                        (default: 21)
  -l LEVEL_OF_THEORY, --level_of_theory=LEVEL_OF_THEORY
                        The level of theory in the format: method_basis
                        (default: blyp_def2-svp)
  --refinement_level_of_theory=R_LEVEL_OF_THEORY
                        The level of theory for geometry refinement in the
                        format: method_basis (default: hf3c_minix)
  --rmsd_value=RMSD_VAL
                        Rmsd geometrical criteria, all structure below this
                        value will not be considered as unique. (default: 0.40
                        angstrom)
  --rmsd_symmetry       Consider the molecular symmetry for the rmsd
                        calculation
  -p PROGRAM, --program=PROGRAM
                        The program to use for this calculation (default:
                        psi4)
  -k KEYWORD_ID, --keyword_id=KEYWORD_ID
                        ID of the QC keywords for the OptimizationDataSet
                        specification (default: None)
  --print_out           Print an output


```

The `SURFACE_MODEL_COLLECTION` refers to the collection 
that contains the set of clusters that span the surface model, in our example there are three water clusters 
of 12 molecules each in the DataSet collection. 

The `SMALL_MOLECULE_COLLECTION` options refers to the collection that contains all the small molecules. 

The `SAMPLING_SHELL` referes to the width of the sampling shell around the cluster. If this number is too large 
it is possible that the molecule will take to long to converge to a binding site during the optimization

**IMPORTANT**: Before running `launch_sampling` you need to spin up two qcfractal-managers, one with a `tag=sampling` and a second with a `tag=refinement` . This module contains two examples of qcfractal-managers running using dask and slurm. For more examples and options check out  [QCFractal manager documentation](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/managers.html)

Finally you can check out the progress of the sampling prodecure by querying the OptimizationDataset asocciated 
to the sampling prodecure. 

In [59]:
import qcfractal.interface as ptl

client = ptl.FractalClient(address="localhost:7777", verify=False)
client.list_collections()

tagline
collection          name                       
OptimizationDataset ch3oh_w12_1            None
                    ch3oh_w12_2            None
                    ch3oh_w12_3            None
                    pentamer_collection    None
                    pre_ch3oh_w12_1        None
                    pre_ch3oh_w12_2        None
                    pre_ch3oh_w12_3        None
                    small_molecules        None
                    water_12               None

The naming convention used in BEEP is first there is the name of the molecule to be sampled then the 
name of the water cluster and number of the water cluster unit within the sets of clusters. Therefore in the example of methanol, if we  want the binding sites of the first water cluster that was sampled we need to retrieve the OptimizationDataset of `ch3oh_w12_1`.

In [84]:
ds_opt = client.get_collection("OptimizationDataset", "ch3oh_w12_1")
ds_opt.status()

KeyError: "Collection 'OptimizationDataset:ch3oh_w12_1' not found."